# Trabalho Prático 2
**Disciplina:** Inteligência Artificial (COMP0427)<br>
**Turma:** T01 - 2018.2<br>
**Orientador:** Prof. Dr. Hendrik Teixeira Macedo<br>
**Equipe:** *Ontological Entity*<br>
**Integrantes:** Airton Matheus Cardoso Leite, Eric Rocha Soares, Hugo Vinicius Dantas Barreto,<br>Igor de Figueiredo Rodrigues, Lucas Brabec Barreto Santana e Tiago Conceição dos Santos

## 1. Descrição do problema

O web site oficial da ferramenta Netica da Norsys disponibiliza diversas redes bayesianas já definidas para download (www.norsys.com/netlibrary/index.htm) em diversas categorias de conhecimento especialista: Ambiental, Médico, Financeiro/Gerencial, Geológico, Diagnóstico Industrial e Outros Industriais. Para cada uma dessas categorias, existem várias redes de exemplo com respectiva descrição da fonte explicativa do problema.

A missão é escolher uma dessas redes e proceder com um experimento em Aprendizado de Máquina, descrito a seguir:<br>
**1-** Eleger uma característica de classe;<br>
**2-** Utilizar técnica de amostragem em redes bayesianas para produzir um dataset de exemplos;<br>
**3-** Reservar parte do dataset para treinamento e parte para testes;<br>
**4-** Treinar modelo Naïve Bayes;<br>
**5-** Treinar modelo de Árvores de Decisão (ou Floresta Aleatória);<br>
**6-** Testar modelos com o conjunto de testes;<br>
**7-** Responder a algumas perguntas: O quanto cada modelo se aproxima da rede bayesiana original? O quanto o tamanho do dataset gerado influencia na qualidade? Smoothing se faz necessário? Quem possui melhor comportamento: Naïve Bayes ou Árvore de Decisão?<br>
**8-** Concluir o experimento.<br>

## 2. Solução adotada

A rede de crença bayesiana adotada, relativa ao conhecimento especialista na área médica de Pneumologia, é a que segue:<br>

![*](Chest_Clinic_Bayesian_Network.png)<br>

<br>É uma versão simplificada de uma rede que poderia ser usada para diagnosticar pacientes que chegam a uma clínica. Cada nó na rede corresponde a alguma condição do paciente. Os dois nós superiores são para predisposições que influenciam a probabilidade das doenças. Essas doenças aparecem na linha abaixo deles. Na parte inferior são sintomas das doenças. Em grande medida, os elos da rede correspondem a causação. Os links entre os nós indicam como os relacionamentos entre os nós são estruturados. Essa é uma estrutura comum para redes de diagnóstico: nós de predisposição no topo, com links para nós representando condições internas e estados de falha, que por sua vez possuem links para nós para evidenciáveis.

**1-** A característica de classe adotada foi a variável aleatória ***Tuberculosis or Cancer***, por seu tom mais dramático.

**2-** A técnica de amostragem implementada foi a ***Prior Sampling***, gerando um *dataset* inicial com 100 mil *samples*.<br>Essa abordagem é melhor explorada na seção 3.

**3-** Foi implementado script de *splitting* do dataset inicial em 66,6% para *training dataset* e 33,4% para *testing dataset*.<br>Essa abordagem é melhor explorada na seção 4.

**4-** Um modelo ***Naïve Bayes*** *baseado em distribuião gaussiana* foi inicialmente adotado. Porém a intenção é ajustá-lo para um classificador de modelos multivariados de Bernoulli, devido à natureza booleana das probabilidades da classe. Essa abordagem é melhor explorada na seção 5.

**5-** Um modelo de Floresta de Decisão Aleatória com Árvores de Decisão ensacadas com variância.<br>Essa abordagem é melhor explorada na seção 6.

**6-** Ambos os modelos foram testados com respectivos testing datasets de 10 mil, 100 mil e 1 milhão de samples. Cada bateria de teste segue nas seções 7 e 8, para Naïve Bayes e Random Forest respectivamente.

**7-** Uma breve discussão dos resultados dos testes é discorrida na seção 9.<br>
(O quanto o tamanho do dataset gerado influencia na qualidade? Smoothing se faz necessário? Quem possui melhor comportamento: Naïve Bayes ou Árvore de Decisão?)

**8-** Nossas considerações finais encerram este projeto na seção 10, seguidas pelas Referências bibliográficas.

## 3. Amostragem de dados

Os modelos probabilísticos usados são geralmente bem complexos e bastante esforço de pesquisa científica em ML é gasto no desenvolvimento de algoritmos que geram soluções aproximadas para o problema de inferência. Um método de amostragem produz respostas ao gerar repetidamente números aleatórios a partir de uma distribuição de interesse e pode ser usado para realizar consultas de inferência marginais ou de probabilidade máxima a posteriori.

O processo de amostragem prévia (Prior Sampling) é consistente e gera amostras com probabilidade: **¹Πⁿ P( Xi | Pais(Xi) )**.

In [46]:
# A node in the bayes network
class Node:
    def __init__(self, n, pa, pr):
        self.name = n
        self.parents = pa
        self.probs = pr
        self.value = None

    '''
    Returns conditional probability of value "true" for the current
    node based on the values of the parent node/s.
    '''
    def conditionalProbability(self):
        index = 0

        for i, p in enumerate(self.parents):
            if p.value == False:
                index += 2 ** (len(self.parents) - i - 1)
        return self.probs[index]

In [47]:
from Node import Node
import random

PLAYING_EXAMPLES = [(True, False), (True, False), (True, True),
                    (True, False), (True, False), (False, True),
                    (False, False), (False, True), (False, True),
                    (False, True), (False, False), (False, True),
                    (False, True)]

class BayesNet:
    # The nodes in the network
    nodes = []

    # Build the initial network
    def __init__(self):
        self.nodes.append(Node("Visit to Asia", [], [0.01])) #index=0
        self.nodes.append(Node("Smoking", [], [0.5])) #index=1


        '''
                Visit Asia+ |   Visit asia-
            T    0.5                0.1
        '''
        self.nodes.append(Node("Tuberculosis",
                               [self.nodes[0]],
                               [0.5, 0.1])) #index=2

        '''
                Lung Cancer+ |   Lung Cancer-
        Smoking    0.10                0.01
        '''
        self.nodes.append(Node("Lung Cancer",
                               [self.nodes[1]],
                               [0.1, 0.01])) #index=3
        '''
                Bronchitis+ |   Bronchitis-
        Smoking    0.6                0.3
        '''
        self.nodes.append(Node("Bronchitis",
                               [self.nodes[1]],
                               [0.6, 0.3])) #index=4
        '''
            Tuberculosis | Lung Cancer | Tuberculosis or Cancer = True |  Tuberculosis or Cancer = False
                True            True                    1                               0
                True            False                   1                               0
                False           True                    1                               0
                False           False                   0                               1

        '''
        self.nodes.append(Node("Tuberculosis or Cancer",
                               [self.nodes[2], self.nodes[3]],
                               [1, 1, 1, 0])) #index=5


        '''
                Tuberculosis or Cancer+ | Tuberculosis or Cancer-
         XRay           0.98                        0.05
        '''
        self.nodes.append(Node("X-Ray Result", [self.nodes[5]], [0.98, 0.05])) #index=6

        '''
                    Tuberculosis or Cancer | Bronchitis | Dyspnea=True | Dyspnea=False
                            True            True               0.90             0.1
                            True            False              0.70             0.3
                            False           True               0.80             0.2
                            False           False              0.10             0.90


        '''
        self.nodes.append(Node("Dyspnea", [self.nodes[5], self.nodes[4]],
                          [0.90,0.70,0.80,0.10]))
        self.nodes.append(self.nodes.pop(5))

    # Prints the current state of the network to stdout
    def printState(self):
        strings = []
        for node in self.nodes:
            strings.append(node.name + " = " + str(node.value))

        print(", ".join(strings))

    def calculatePlayOutsideProbabilities(self, rainingInstances):
        playing = [0, 0]
        total = [0, 0]
        prob = [0.0, 0.0]

        for sample in rainingInstances:
            if sample[0]:
                playing[0] += 1 if sample[1] else 0
                total[0] += 1
            else:
                playing[1] += 1 if sample[1] else 0
                total[1] += 1

        prob[0] = float(playing[0]) / float(total[0])
        prob[1] = float(playing[1]) / float(total[1])

        return prob

    '''
    This method will sample the value for a node given its
    conditional probability.
    '''
    def sampleNode(self, node):
        node.value = True if random.random() <= node.conditionalProbability() else False

    '''
    This method assigns new values to the nodes in the network by
    sampling from the joint distribution.  Based on the PRIOR-SAMPLE
    from the text book/slides
    '''
    def priorSample(self):
        for n in self.nodes:
            self.sampleNode(n)

        return self.nodes
    '''
    This method will return true if all the evidence variables in the
    network have the value specified by the evidence values.
    '''
    def testModel(self, indicesOfEvidenceNodes, evidenceValues):
        for i in range(len(indicesOfEvidenceNodes)):
            if (self.nodes[indicesOfEvidenceNodes[i]].value != evidenceValues[i]):
                return False

        return True

    def printState(self):
        strings = []
        for node in self.nodes:
            strings.append(node.name + " = " + str(node.value))

        print(", ".join(strings))

    '''
        this method receives an number n of samples that will generate with prior sampling
        the result is an array of maps consisting of key values
        EX:
        [{Visit to Asia:False},{Smoking:False},{Tuberculosis:False},{Lung Cancer:False},{Bronchitis:True},{Tuberculosis or Cancer:False},{X-Ray Result:False},{Dyspnea:True}]
    '''
    def getSamples(self, n):
        result = []
        sublist = []
        for i in range(n):
            for node in self.priorSample():
                sublist.append({node.name: node.value})

            result.append(sublist)
            #self.printState()

        return result

    '''
        this method receives an number n of samples that will generate with prior sampling
        the result is file named sampling.txt created in the folder of tthe project with strings consisting of key values
        EX:
        {Visit to Asia:False},{Smoking:False},{Tuberculosis:False},{Lung Cancer:False},{Bronchitis:True},{Tuberculosis or Cancer:False},{X-Ray Result:False},{Dyspnea:True}

    '''
    def beginSamplingAndSaveToFile(self,n):
        result = []

        for i in range(n):
            strings = []
            for node in self.priorSample():
                strings.append(node.name + ":" + str(node.value))

            self.appendStringToFile("samples.csv", ",".join(strings))

            #self.printState()


        return result

    # helper method that gets a file name and a string and saves the string to the file
    def appendStringToFile(self, filename, input):
        with open(filename, 'a') as file:
                file.write(input + "\n")

if __name__ == "__main__":
    with open("samples.csv",'w'):
        pass
    b = BayesNet() # Creates a bayes net
    num = input("Numero:")
    nodes = b.beginSamplingAndSaveToFile(int(num)) #creates and saves 100 thousands of samples from the net
    print("Concluido.")
    '''
    strings = []
    for node in nodes:
        for colums in node:
            strings.append(colums)

    print(strings)

    '''

ModuleNotFoundError: No module named 'Node'

## 4. Dataset splitting

É padrão em ML se dividir o conjunto de dados em subconjuntos de treinamento e teste. A razão para isso é que tentar avaliar o agente de aprendizado com dados já treinados é algo irrealista, o objetivo de um agente supervisionado é ser capaz de classificar dados previamente desconhecidos.

In [51]:
# program to split the sample into training and testing datasets
import sys

# opening the files
sample = open('./Sampling/samples.csv','r')
#sample = open(sys.argv[1], "r")

training_dataset = open('./datasets/training_dataset.csv','w')
testing_dataset = open('./datasets/testing_dataset.csv','w')


sample_number = len(sample.readlines())
print('Total number of examples = ', sample_number)

training_cut_parameter = int(sample_number * 0.666)
print('Number of training examples = ', training_cut_parameter)
testing_cut_parameter = int(sample_number * 0.334)
print('Number of testing examples = ', testing_cut_parameter)

# return to the sample file start
sample.seek(0)

# splitting the sample into 70% training dataset
for example in range(training_cut_parameter):
    training_dataset.write(sample.readline())

# and 30% testing dataset
for example in range(testing_cut_parameter):
    testing_dataset.write(sample.readline())

# safely closing the files
sample.close()
training_dataset.close()
testing_dataset.close()


Total number of examples =  1000
Number of training examples =  666
Number of testing examples =  334


## 5. Naïve Bayes

O algoritmo *Naïve Bayes* é um classificador probabilístico baseado no **Teorema de Bayes** que desconsidera a correlação entre variáveis, tratando cada fator de forma independente. E leva em consideração apenas a relação de probabilidade condicional a posteriori de cada característica com o atributo de classe.

In [61]:
import csv
import random
import math

def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [str(x).split(':')[1] == 'True' for x in dataset[i]]
    return dataset
 
def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]
 
def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated
 
def mean(numbers):
	return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)
 
def summarize(dataset):
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	del summaries[-1]
	return summaries
 
def summarizeByClass(dataset):
	separated = separateByClass(dataset)
	summaries = {}
	for classValue, instances in separated.items():
		summaries[classValue] = summarize(instances)
	return summaries
 
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
 
def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev)
	return probabilities
			
def predict(summaries, inputVector):
	probabilities = calculateClassProbabilities(summaries, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel
 
def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
	return predictions
 
def getAccuracy(testSet, predictions):
	correct = 0
	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0


### 5.1. Leitura dos dados:

In [62]:
filename = './Sampling/samples.csv'
splitRatio = 0.666
dataset = loadCsv(filename)
trainingSet, testSet = splitDataset(dataset, splitRatio)
print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))

Split 1000 rows into train=666 and test=334 rows


### 5.2. Aprendendo as probabilidades com Gaussian Naïve Bayes:

In [63]:
summaries = summarizeByClass(trainingSet)
print('Summary by class value: {0}'.format(summaries))

Summary by class value: {True: [(0.0121580547112462, 0.10975815683276703), (0.6048632218844985, 0.4896247282637922), (0.16109422492401215, 0.368177805662427), (0.10638297872340426, 0.30879681803953996), (0.790273556231003, 0.40773345616996076), (0.2674772036474164, 0.4433176133862686), (0.3130699088145897, 0.4644489240214539)], False: [(0.008902077151335312, 0.09406959443339322), (0.4124629080118694, 0.4930096341494111), (0.026706231454005934, 0.16146321191014087), (0.01483679525222552, 0.1210791754899165), (0.1543026706231454, 0.3617758001116408), (0.04154302670623145, 0.19983920341257794), (0.09792284866468842, 0.2976522519276067)]}


### 5.3. Predições para todos os exemplos do conjunto de treinamento:

In [64]:
predictions = getPredictions(summaries, testSet)
print('Predictions: {0}'.format(predictions)) 

Predictions: [False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, True, True, False, True, False, False, False, True, False, False, True, False, True, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, True, True, False, False, False, False, False, True, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, True, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, True, True, False, False, False, False, True, False, True, False, False, True, Fal

### 5.4. Calculando a acurácia do modelo treinado:

In [60]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: {0}%'.format(accuracy))

Accuracy: 79.94011976047905%


## 6. Random Decision Forest

Como é natural para ambientes de conflito entre dois jogadores com interesses opostos *(competição pura)*, o jogo é de soma 

## 7. Bateria de testes para Naïve Bayes

Como é natural para ambientes de conflito entre dois jogadores com interesses opostos *(competição pura)*, o jogo é de soma 

## 8. Bateria de testes para Random Decision Forests

Como é natural para ambientes de conflito entre dois jogadores com interesses opostos *(competição pura)*, o jogo é de soma 

## 9. Discussão dos resultados

Como é natural para ambientes de conflito entre dois jogadores com interesses opostos *(competição pura)*, o jogo é de soma 

## 10. Considerações finais

Como é natural para ambientes de conflito entre dois jogadores com interesses opostos *(competição pura)*, o jogo é de soma 

## Referências

Lauritzen, Steffen L. and David J. Spiegelhalter (1988) "Local computations with probabilities on graphical structures and their application to expert systems" in J. Royal Statistics Society B, 50(2), 157-194. Online em: https://www.eecis.udel.edu/~shatkay/Course/papers/Lauritzen1988.pdf


In [ ]:
def minimax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minimax_decision:
    return argmax(game.actions(state),
                  key=lambda a: min_value(game.result(state, a)))

In [ ]:
#poda 𝛼-β

def alphabeta_cutoff_search(state, game, d=1, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""
    eval_fn = game.eval_function
    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = -infinity
        for a in game.actions(state):
            st = game.result(state, a)
            #game.display(st)
            v = max(v, min_value(st, alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = infinity
        for a in game.actions(state):
            st = game.result(state, a)
            #game.display(st)
            v = min(v, max_value(st, alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state, player : game.utility(state, player))
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        st = game.result(state, a)
        #game.display(st)
        v = min_value(st, best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action



In [1]:
# riddles.io, 2019
# classe que implementa uma unidade de célula

class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return '{},{}'.format(self.x, self.y)
    

In [5]:
# riddles.io, 2019
# classe que implementa o campo do jogo

from field.point import Point


class Field:

    def __init__(self):
        self.width = None
        self.height = None
        self.cells = None

    def parse(self, field_input):
        self.cells = [[] for _ in range(self.width)]
        x = 0
        y = 0

        for cell in field_input.split(','):
            self.cells[x].insert(y, cell)
            x += 1

            if x == self.width:
                x = 0
                y += 1

    def get_cell_mapping(self):
        cell_map = {}

        for x in range(self.width):
            for y in range(self.height):
                cell = self.cells[x][y]
                cell_type_list = cell_map.get(cell, [])
                cell_type_list.append(Point(x, y))
                cell_map[cell] = cell_type_list

        return cell_map
    

In [6]:
# riddles.io, 2019
# classe que implementa os tipos possíveis de ações permitidas para os jogadores

from enum import Enum


class MoveType(Enum):
    KILL = 'kill'
    BIRTH = 'birth'
    PASS = 'pass'

    def __str__(self):
        return self.value
    

In [7]:
# riddles.io, 2019
# classe que implementa a execução das ações em si

from move.move_type import MoveType


class Move:

    def __init__(self, move_type, target_point=None, sacrifice_points=None):
        self.move_type = move_type
        self.target_point = target_point
        self.sacrifice_points = sacrifice_points

    def __str__(self):
        if self.move_type == MoveType.KILL:
            return '{} {}'.format(self.move_type, self.target_point)
        elif self.move_type == MoveType.BIRTH:
            sacrifice_string = ' '.join(str(p) for p in self.sacrifice_points)
            return '{} {} {}'.format(self.move_type, self.target_point, sacrifice_string)

        return str(MoveType.PASS)


In [8]:
# riddles.io, 2019
# classe que implementa o jogador

class Player:

    def __init__(self, name):
        self.name = name
        self.move = None
        self.id = None
        self.living_cells = -1
        self.previous_move = None


In [9]:
# riddles.io, 2019
# classe que implementa a configuração de jogo

from sys import stdin, stdout, stderr
import traceback
import time

from bot.player import Player
from field.field import Field


class Game:
    def __init__(self):
        self.time_per_move = -1
        self.timebank = -1
        self.last_update = None
        self.max_rounds = -1

        self.round = 0
        self.player_names = []
        self.players = {}
        self.me = None
        self.opponent = None
        self.field = Field()

    def update(self, data):
        # start timer
        self.last_update = time.time()
        for line in data.split('\n'):

            line = line.strip()
            if len(line) <= 0:
                continue

            tokens = line.split()
            if tokens[0] == "settings":
                self.parse_settings(tokens[1], tokens[2])
            elif tokens[0] == "update":
                if tokens[1] == "game":
                    self.parse_game_updates(tokens[2], tokens[3])
                else:
                    self.parse_player_updates(tokens[1], tokens[2], tokens[3])
            elif tokens[0] == "action":
                self.timebank = int(tokens[2])
                # Launching bot logic happens after setup finishes

    def parse_settings(self, key, value):
        if key == "timebank":
            self.timebank = int(value)
        elif key == "time_per_move":
            self.time_per_move = int(value)
        elif key == "player_names":
            self.player_names = value.split(',')
            self.players = {name: Player(name) for name in self.player_names}
        elif key == "your_bot":
            self.me = self.players[value]
            self.opponent = self.players[[name for name in self.player_names if name != value][0]]
        elif key == "your_botid":
            self.me.id = value
            self.opponent.id = str(2 - (int(value) + 1))
        elif key == "field_width":
            self.field.width = int(value)
        elif key == "field_height":
            self.field.height = int(value)
        elif key == "max_rounds":
            self.max_rounds = int(value)
        else:
            stderr.write('Cannot parse settings input with key {}'.format(key))

    def parse_game_updates(self, key, value):
        if key == "round":
            self.round = int(value)
        elif key == "field":
            self.field.parse(value)
        else:
            stderr.write('Cannot parse game update with key {}'.format(key))

    def parse_player_updates(self, player_name, key, value):
        player = self.players.get(player_name)

        if player is None:
            stderr.write('Cannot find player with name {}'.format(player_name))
            return

        if key == "living_cells":
            player.living_cells = int(value)
        elif key == "move":
            player.previous_move = value
        else:
            stderr.write('Cannot parse {} update with key {}'.format(player_name, key))

    def time_remaining(self):
        return self.timebank - int(1000 * (time.clock() - self.last_update))

    @staticmethod
    def print_move(move):
        """issue an order"""
        stdout.write('{}\n'.format(move))
        stdout.flush()

    def run(self, bot):
        """parse input, update game state and call the bot classes do_turn method"""
        not_finished = True
        data = ''

        while not stdin.closed and not_finished:
            try:
                current_line = stdin.readline().rstrip('\r\n')

                if len(current_line) <= 0:
                    time.sleep(1)
                    continue

                data += current_line + "\n"
                if current_line.lower().startswith("action"):
                    self.update(data)

                    move = bot.make_move(self)
                    self.print_move(move)

                    data = ''
                elif current_line.lower().startswith("quit"):
                    not_finished = False
            except EOFError:
                break
            except KeyboardInterrupt:
                raise
            except:
                # don't raise error or return so that bot attempts to stay alive
                traceback.print_exc(file=stderr)
                stderr.flush()


In [10]:
# riddles.io, 2019
# classe que implementa o robô jogador

import random

from move.move import Move
from move.move_type import MoveType


class Bot:

    def __init__(self):
        random.seed()  # set seed here if needed

    def make_move(self, game):
        """
        Performs a Birth or a Kill move, currently returns a random move.
        Implement this method to make the bot smarter.
        """
        cell_map = game.field.get_cell_mapping()

        # if random.random() < 0.5:
        return self.make_random_birth_move(game, cell_map)

        # return self.make_random_kill_move(game, cell_map)

    def make_random_birth_move(self, game, cell_map):
        dead_cells = cell_map.get('.', [])
        my_cells = cell_map.get(game.me.id, [])

        if len(dead_cells) <= 0 or len(my_cells) < 2:
            return self.make_random_kill_move(game, cell_map)

        random_birth = dead_cells[random.randrange(len(dead_cells))]

        random_sacrifices = []
        for i in range(2):
            random_sacrifice = my_cells.pop(random.randrange(len(my_cells)))
            random_sacrifices.append(random_sacrifice)

        return Move(MoveType.BIRTH, random_birth, random_sacrifices)

    def make_random_kill_move(self, game, cell_map):
        my_cells = cell_map.get(game.me.id, [])
        opponent_cells = cell_map.get(game.opponent.id, [])
        living_cells = my_cells + opponent_cells

        if len(living_cells) <= 0:
            return Move(MoveType.PASS)

        random_kill = living_cells[random.randrange(len(living_cells))]

        return Move(MoveType.KILL, random_kill)


In [11]:
# Hendrik Macedo, 2017
# módulo que implementa diversas utilidades auxiliares como estruturas de dados e funções

"""Provides some utilities widely used by other modules"""

import bisect
import collections
import collections.abc
import operator
import os.path
import random
import math
import functools

# ______________________________________________________________________________
# Functions on Sequences and Iterables


def sequence(iterable):
    """Coerce iterable to sequence, if it is not already one."""
    return (iterable if isinstance(iterable, collections.abc.Sequence)
            else tuple(iterable))


def removeall(item, seq):
    """Return a copy of seq (or string) with all occurences of item removed."""
    if isinstance(seq, str):
        return seq.replace(item, '')
    else:
        return [x for x in seq if x != item]


def unique(seq):  # TODO: replace with set
    """Remove duplicate elements from seq. Assumes hashable elements."""
    return list(set(seq))


def count(seq):
    """Count the number of items in sequence that are interpreted as true."""
    return sum(bool(x) for x in seq)


def product(numbers):
    """Return the product of the numbers, e.g. product([2, 3, 10]) == 60"""
    result = 1
    for x in numbers:
        result *= x
    return result


def first(iterable, default=None):
    """Return the first element of an iterable or the next element of a generator; or default."""
    try:
        return iterable[0]
    except IndexError:
        return default
    except TypeError:
        return next(iterable, default)


def is_in(elt, seq):
    """Similar to (elt in seq), but compares with 'is', not '=='."""
    return any(x is elt for x in seq)


def mode(data):
    """Return the most common data item. If there are ties, return any one of them."""
    [(item, count)] = collections.Counter(data).most_common(1)
    return item


# ______________________________________________________________________________
# argmin and argmax


identity = lambda x: x

argmin = min
argmax = max


def argmin_random_tie(seq, key=identity):
    """Return a minimum element of seq; break ties at random."""
    return argmin(shuffled(seq), key=key)


def argmax_random_tie(seq, key=identity):
    """Return an element with highest fn(seq[i]) score; break ties at random."""
    return argmax(shuffled(seq), key=key)


def shuffled(iterable):
    """Randomly shuffle a copy of iterable."""
    items = list(iterable)
    random.shuffle(items)
    return items


# ______________________________________________________________________________
# Statistical and mathematical functions


def histogram(values, mode=0, bin_function=None):
    """Return a list of (value, count) pairs, summarizing the input values.
    Sorted by increasing value, or if mode=1, by decreasing count.
    If bin_function is given, map it over values first."""
    if bin_function:
        values = map(bin_function, values)

    bins = {}
    for val in values:
        bins[val] = bins.get(val, 0) + 1

    if mode:
        return sorted(list(bins.items()), key=lambda x: (x[1], x[0]),
                      reverse=True)
    else:
        return sorted(bins.items())


def dotproduct(X, Y):
    """Return the sum of the element-wise product of vectors X and Y."""
    return sum(x * y for x, y in zip(X, Y))


def element_wise_product(X, Y):
    """Return vector as an element-wise product of vectors X and Y"""
    assert len(X) == len(Y)
    return [x * y for x, y in zip(X, Y)]


def matrix_multiplication(X_M, *Y_M):
    """Return a matrix as a matrix-multiplication of X_M and arbitary number of matrices *Y_M"""

    def _mat_mult(X_M, Y_M):
        """Return a matrix as a matrix-multiplication of two matrices X_M and Y_M
        >>> matrix_multiplication([[1, 2, 3],
                                   [2, 3, 4]],
                                   [[3, 4],
                                    [1, 2],
                                    [1, 0]])
        [[8, 8],[13, 14]]
        """
        assert len(X_M[0]) == len(Y_M)

        result = [[0 for i in range(len(Y_M[0]))] for j in range(len(X_M))]
        for i in range(len(X_M)):
            for j in range(len(Y_M[0])):
                for k in range(len(Y_M)):
                    result[i][j] += X_M[i][k] * Y_M[k][j]
        return result

    result = X_M
    for Y in Y_M:
        result = _mat_mult(result, Y)

    return result


def vector_to_diagonal(v):
    """Converts a vector to a diagonal matrix with vector elements
    as the diagonal elements of the matrix"""
    diag_matrix = [[0 for i in range(len(v))] for j in range(len(v))]
    for i in range(len(v)):
        diag_matrix[i][i] = v[i]

    return diag_matrix


def vector_add(a, b):
    """Component-wise addition of two vectors."""
    return tuple(map(operator.add, a, b))


def scalar_vector_product(X, Y):
    """Return vector as a product of a scalar and a vector"""
    return [X * y for y in Y]


def scalar_matrix_product(X, Y):
    """Return matrix as a product of a scalar and a matrix"""
    return [scalar_vector_product(X, y) for y in Y]


def inverse_matrix(X):
    """Inverse a given square matrix of size 2x2"""
    assert len(X) == 2
    assert len(X[0]) == 2
    det = X[0][0] * X[1][1] - X[0][1] * X[1][0]
    assert det != 0
    inv_mat = scalar_matrix_product(1.0/det, [[X[1][1], -X[0][1]], [-X[1][0], X[0][0]]])

    return inv_mat


def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)


def weighted_sample_with_replacement(n, seq, weights):
    """Pick n samples from seq at random, with replacement, with the
    probability of each element in proportion to its corresponding
    weight."""
    sample = weighted_sampler(seq, weights)

    return [sample() for _ in range(n)]


def weighted_sampler(seq, weights):
    """Return a random-sample function that picks from seq weighted by weights."""
    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)

    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]


def rounder(numbers, d=4):
    """Round a single number, or sequence of numbers, to d decimal places."""
    if isinstance(numbers, (int, float)):
        return round(numbers, d)
    else:
        constructor = type(numbers)     # Can be list, set, tuple, etc.
        return constructor(rounder(n, d) for n in numbers)


def num_or_str(x):
    """The argument is a string; convert to a number if
       possible, or strip it."""
    try:
        return int(x)
    except ValueError:
        try:
            return float(x)
        except ValueError:
            return str(x).strip()


def normalize(dist):
    """Multiply each number by a constant such that the sum is 1.0"""
    if isinstance(dist, dict):
        total = sum(dist.values())
        for key in dist:
            dist[key] = dist[key] / total
            assert 0 <= dist[key] <= 1, "Probabilities must be between 0 and 1."
        return dist
    total = sum(dist)
    return [(n / total) for n in dist]


def clip(x, lowest, highest):
    """Return x clipped to the range [lowest..highest]."""
    return max(lowest, min(x, highest))


def sigmoid_derivative(value):
    return value * (1 - value)


def sigmoid(x):
    """Return activation value of x with sigmoid function"""
    return 1/(1 + math.exp(-x))


def step(x):
    """Return activation value of x with sign function"""
    return 1 if x >= 0 else 0


def gaussian(mean, st_dev, x):
    """Given the mean and standard deviation of a distribution, it returns the probability of x."""
    return 1/(math.sqrt(2*math.pi)*st_dev)*math.e**(-0.5*(float(x-mean)/st_dev)**2)


try:  # math.isclose was added in Python 3.5; but we might be in 3.4
    from math import isclose
except ImportError:
    def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
        """Return true if numbers a and b are close to each other."""
        return abs(a - b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)


# ______________________________________________________________________________
# Grid Functions


orientations = EAST, NORTH, WEST, SOUTH = [(1, 0), (0, 1), (-1, 0), (0, -1)]
turns = LEFT, RIGHT = (+1, -1)


def turn_heading(heading, inc, headings=orientations):
    return headings[(headings.index(heading) + inc) % len(headings)]


def turn_right(heading):
    return turn_heading(heading, RIGHT)


def turn_left(heading):
    return turn_heading(heading, LEFT)


def distance(a, b):
    """The distance between two (x, y) points."""
    xA, yA = a
    xB, yB = b
    return math.hypot((xA - xB), (yA - yB))


def distance_squared(a, b):
    """The square of the distance between two (x, y) points."""
    xA, yA = a
    xB, yB = b
    return (xA - xB)**2 + (yA - yB)**2


def vector_clip(vector, lowest, highest):
    """Return vector, except if any element is less than the corresponding
    value of lowest or more than the corresponding value of highest, clip to
    those values."""
    return type(vector)(map(clip, vector, lowest, highest))


# ______________________________________________________________________________
# Misc Functions


def memoize(fn, slot=None, maxsize=32):
    """Memoize fn: make it remember the computed value for any argument list.
    If slot is specified, store result in that slot of first argument.
    If slot is false, use lru_cache for caching the values."""
    if slot:
        def memoized_fn(obj, *args):
            if hasattr(obj, slot):
                return getattr(obj, slot)
            else:
                val = fn(obj, *args)
                setattr(obj, slot, val)
                return val
    else:
        @functools.lru_cache(maxsize=maxsize)
        def memoized_fn(*args):
            return fn(*args)

    return memoized_fn


def name(obj):
    """Try to find some reasonable name for the object."""
    return (getattr(obj, 'name', 0) or getattr(obj, '__name__', 0) or
            getattr(getattr(obj, '__class__', 0), '__name__', 0) or
            str(obj))


def isnumber(x):
    """Is x a number?"""
    return hasattr(x, '__int__')


def issequence(x):
    """Is x a sequence?"""
    return isinstance(x, collections.abc.Sequence)


def print_table(table, header=None, sep='   ', numfmt='{}'):
    """Print a list of lists as a table, so that columns line up nicely.
    header, if specified, will be printed as the first row.
    numfmt is the format for all numbers; you might want e.g. '{:.2f}'.
    (If you want different formats in different columns,
    don't use print_table.) sep is the separator between columns."""
    justs = ['rjust' if isnumber(x) else 'ljust' for x in table[0]]

    if header:
        table.insert(0, header)

    table = [[numfmt.format(x) if isnumber(x) else x for x in row]
             for row in table]

    sizes = list(
            map(lambda seq: max(map(len, seq)),
                list(zip(*[map(str, row) for row in table]))))

    for row in table:
        print(sep.join(getattr(
            str(x), j)(size) for (j, size, x) in zip(justs, sizes, row)))


def AIMAFile(components, mode='r'):
    """Open a file based at the AIMA root directory."""
    aima_root = os.path.dirname(__file__)

    aima_file = os.path.join(aima_root, *components)

    return open(aima_file)


def DataFile(name, mode='r'):
    "Return a file in the AIMA /aima-data directory."
    return AIMAFile(['aima-data', name], mode)


# ______________________________________________________________________________
# Expressions

# See https://docs.python.org/3/reference/expressions.html#operator-precedence
# See https://docs.python.org/3/reference/datamodel.html#special-method-names

class Expr(object):
    """A mathematical expression with an operator and 0 or more arguments.
    op is a str like '+' or 'sin'; args are Expressions.
    Expr('x') or Symbol('x') creates a symbol (a nullary Expr).
    Expr('-', x) creates a unary; Expr('+', x, 1) creates a binary."""

    def __init__(self, op, *args):
        self.op = str(op)
        self.args = args

    # Operator overloads
    def __neg__(self):
        return Expr('-', self)

    def __pos__(self):
        return Expr('+', self)

    def __invert__(self):
        return Expr('~', self)

    def __add__(self, rhs):
        return Expr('+', self, rhs)

    def __sub__(self, rhs):
        return Expr('-', self, rhs)

    def __mul__(self, rhs):
        return Expr('*', self, rhs)

    def __pow__(self, rhs):
        return Expr('**', self, rhs)

    def __mod__(self, rhs):
        return Expr('%', self, rhs)

    def __and__(self, rhs):
        return Expr('&', self, rhs)

    def __xor__(self, rhs):
        return Expr('^', self, rhs)

    def __rshift__(self, rhs):
        return Expr('>>', self, rhs)

    def __lshift__(self, rhs):
        return Expr('<<', self, rhs)

    def __truediv__(self, rhs):
        return Expr('/', self, rhs)

    def __floordiv__(self, rhs):
        return Expr('//', self, rhs)

    def __matmul__(self, rhs):
        return Expr('@', self, rhs)

    def __or__(self, rhs):
        """Allow both P | Q, and P |'==>'| Q."""
        if isinstance(rhs, Expression):
            return Expr('|', self, rhs)
        else:
            return PartialExpr(rhs, self)

    # Reverse operator overloads
    def __radd__(self, lhs):
        return Expr('+', lhs, self)

    def __rsub__(self, lhs):
        return Expr('-', lhs, self)

    def __rmul__(self, lhs):
        return Expr('*', lhs, self)

    def __rdiv__(self, lhs):
        return Expr('/', lhs, self)

    def __rpow__(self, lhs):
        return Expr('**', lhs, self)

    def __rmod__(self, lhs):
        return Expr('%', lhs, self)

    def __rand__(self, lhs):
        return Expr('&', lhs, self)

    def __rxor__(self, lhs):
        return Expr('^', lhs, self)

    def __ror__(self, lhs):
        return Expr('|', lhs, self)

    def __rrshift__(self, lhs):
        return Expr('>>', lhs, self)

    def __rlshift__(self, lhs):
        return Expr('<<', lhs, self)

    def __rtruediv__(self, lhs):
        return Expr('/', lhs, self)

    def __rfloordiv__(self, lhs):
        return Expr('//', lhs, self)

    def __rmatmul__(self, lhs):
        return Expr('@', lhs, self)

    def __call__(self, *args):
        "Call: if 'f' is a Symbol, then f(0) == Expr('f', 0)."
        if self.args:
            raise ValueError('can only do a call for a Symbol, not an Expr')
        else:
            return Expr(self.op, *args)

    # Equality and repr
    def __eq__(self, other):
        "'x == y' evaluates to True or False; does not build an Expr."
        return (isinstance(other, Expr)
                and self.op == other.op
                and self.args == other.args)

    def __hash__(self): return hash(self.op) ^ hash(self.args)

    def __repr__(self):
        op = self.op
        args = [str(arg) for arg in self.args]
        if op.isidentifier():       # f(x) or f(x, y)
            return '{}({})'.format(op, ', '.join(args)) if args else op
        elif len(args) == 1:        # -x or -(x + 1)
            return op + args[0]
        else:                       # (x - y)
            opp = (' ' + op + ' ')
            return '(' + opp.join(args) + ')'

# An 'Expression' is either an Expr or a Number.
# Symbol is not an explicit type; it is any Expr with 0 args.


Number = (int, float, complex)
Expression = (Expr, Number)


def Symbol(name):
    """A Symbol is just an Expr with no args."""
    return Expr(name)


def symbols(names):
    """Return a tuple of Symbols; names is a comma/whitespace delimited str."""
    return tuple(Symbol(name) for name in names.replace(',', ' ').split())


def subexpressions(x):
    """Yield the subexpressions of an Expression (including x itself)."""
    yield x
    if isinstance(x, Expr):
        for arg in x.args:
            yield from subexpressions(arg)


def arity(expression):
    """The number of sub-expressions in this expression."""
    if isinstance(expression, Expr):
        return len(expression.args)
    else:  # expression is a number
        return 0

# For operators that are not defined in Python, we allow new InfixOps:


class PartialExpr:
    """Given 'P |'==>'| Q, first form PartialExpr('==>', P), then combine with Q."""
    def __init__(self, op, lhs):
        self.op, self.lhs = op, lhs

    def __or__(self, rhs):
        return Expr(self.op, self.lhs, rhs)

    def __repr__(self):
        return "PartialExpr('{}', {})".format(self.op, self.lhs)


def expr(x):
    """Shortcut to create an Expression. x is a str in which:
    - identifiers are automatically defined as Symbols.
    - ==> is treated as an infix |'==>'|, as are <== and <=>.
    If x is already an Expression, it is returned unchanged. Example:
    >>> expr('P & Q ==> Q')
    ((P & Q) ==> Q)
    """
    if isinstance(x, str):
        return eval(expr_handle_infix_ops(x), defaultkeydict(Symbol))
    else:
        return x


infix_ops = '==> <== <=>'.split()


def expr_handle_infix_ops(x):
    """Given a str, return a new str with ==> replaced by |'==>'|, etc.
    >>> expr_handle_infix_ops('P ==> Q')
    "P |'==>'| Q"
    """
    for op in infix_ops:
        x = x.replace(op, '|' + repr(op) + '|')
    return x


class defaultkeydict(collections.defaultdict):
    """Like defaultdict, but the default_factory is a function of the key.
    >>> d = defaultkeydict(len); d['four']
    4
    """
    def __missing__(self, key):
        self[key] = result = self.default_factory(key)
        return result


class hashabledict(dict):
    """Allows hashing by representing a dictionary as tuple of key:value pairs
       May cause problems as the hash value may change during runtime
    """
    def __tuplify__(self):
        return tuple(sorted(self.items()))

    def __hash__(self):
        return hash(self.__tuplify__())

    def __lt__(self, odict):
        assert isinstance(odict, hashabledict)
        return self.__tuplify__() < odict.__tuplify__()

    def __gt__(self, odict):
        assert isinstance(odict, hashabledict)
        return self.__tuplify__() > odict.__tuplify__()

    def __le__(self, odict):
        assert isinstance(odict, hashabledict)
        return self.__tuplify__() <= odict.__tuplify__()

    def __ge__(self, odict):
        assert isinstance(odict, hashabledict)
        return self.__tuplify__() >= odict.__tuplify__()


# ______________________________________________________________________________
# Queues: Stack, FIFOQueue, PriorityQueue

# TODO: queue.PriorityQueue
# TODO: Priority queues may not belong here -- see treatment in search.py


class Queue:

    """Queue is an abstract class/interface. There are three types:
        Stack(): A Last In First Out Queue.
        FIFOQueue(): A First In First Out Queue.
        PriorityQueue(order, f): Queue in sorted order (default min-first).
    Each type supports the following methods and functions:
        q.append(item)  -- add an item to the queue
        q.extend(items) -- equivalent to: for item in items: q.append(item)
        q.pop()         -- return the top item from the queue
        len(q)          -- number of items in q (also q.__len())
        item in q       -- does q contain item?
    Note that isinstance(Stack(), Queue) is false, because we implement stacks
    as lists.  If Python ever gets interfaces, Queue will be an interface."""

    def __init__(self):
        raise NotImplementedError

    def extend(self, items):
        for item in items:
            self.append(item)


def Stack():
    """Return an empty list, suitable as a Last-In-First-Out Queue."""
    return []


class FIFOQueue(Queue):

    """A First-In-First-Out Queue."""

    def __init__(self, maxlen=None, items=[]):
        self.queue = collections.deque(items, maxlen)

    def append(self, item):
        if not self.queue.maxlen or len(self.queue) < self.queue.maxlen:
            self.queue.append(item)
        else:
            raise Exception('FIFOQueue is full')

    def extend(self, items):
        if not self.queue.maxlen or len(self.queue) + len(items) <= self.queue.maxlen:
            self.queue.extend(items)
        else:
            raise Exception('FIFOQueue max length exceeded')

    def pop(self):
        if len(self.queue) > 0:
            return self.queue.popleft()
        else:
            raise Exception('FIFOQueue is empty')

    def __len__(self):
        return len(self.queue)

    def __contains__(self, item):
        return item in self.queue


class PriorityQueue(Queue):

    """A queue in which the minimum (or maximum) element (as determined by f and
    order) is returned first. If order is min, the item with minimum f(x) is
    returned first; if order is max, then it is the item with maximum f(x).
    Also supports dict-like lookup."""

    def __init__(self, order=min, f=lambda x: x):
        self.A = []
        self.order = order
        self.f = f

    def append(self, item):
        bisect.insort(self.A, (self.f(item), item))

    def __len__(self):
        return len(self.A)

    def pop(self):
        if self.order == min:
            return self.A.pop(0)[1]
        else:
            return self.A.pop()[1]

    def __contains__(self, item):
        return any(item == pair[1] for pair in self.A)

    def __getitem__(self, key):
        for _, item in self.A:
            if item == key:
                return item

    def __delitem__(self, key):
        for i, (value, item) in enumerate(self.A):
            if item == key:
                self.A.pop(i)


# ______________________________________________________________________________
# Useful Shorthands


class Bool(int):
    """Just like `bool`, except values display as 'T' and 'F' instead of 'True' and 'False'"""
    __str__ = __repr__ = lambda self: 'T' if self else 'F'


T = Bool(True)
F = Bool(False)


In [12]:
# Hendrik Macedo, 2017
# funções que implementam os tipos de jogador

def query_player(game, state):
    """Make a move by querying standard input."""
    print("current state:")
    game.display(state)
    print("available moves: {}".format(game.actions(state)))
    print("")
    move_string = input('Your move? ')
    try:
        move = eval(move_string)
    except NameError:
        move = move_string
    return move


def random_player(game, state):
    """A player that chooses a legal move at random."""
    return random.choice(game.actions(state))

def alphabeta_player(game, state):
    return alphabeta_search(state, game)

In [13]:
# Hendrik Macedo, 2017
# módulo que implementa os algoritmos para árvore minimax com poda alpha-beta

from collections import namedtuple
import random

from utils import argmax
from canvas import Canvas

infinity = float('inf')
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

def minimax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minimax_decision:
    return argmax(game.actions(state),
                  key=lambda a: min_value(game.result(state, a)))


def alphabeta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_cutoff_search:
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


def alphabeta_cutoff_search(state, game, d=4, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""

    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a),
                                 alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a),
                                 alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or
                   (lambda state, depth: depth > d or
                    game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state: game.utility(state, player))
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


ModuleNotFoundError: No module named 'utils'

In [14]:
# Hendrik Macedo, 2017
# classe que implementa a base genérica para um jogo ser instanciado

class Game:
    """A game has a utility for each
    state and a terminal test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))

## 5. Análise e teste

--<br>
Aqui, você deve mostrar o feedback que o ambiente de competição te dará para suas submissões.<br>
Mostrar gráficos de desemepnho comparativo de sua pontuação com a de outros competidores.<br>
Mostrar a evolução de sua pontuação com as versões, mostrar a evolução de performance (tempo de processamento) de seu código com as versões, mostrar screenshots de seu bot agindo em momentos distintos do jogo e/ou link para videos de seu bot são exemplos do que deve conter aqui.<br>--

## 6. Divisão de trabalho e checkpoints

**• Airton:** Estudo do problema, foco nas regras do Game of Life e ações do problema (GoLaD), suporte na escrita do relatório, codificação e testes.<br>
**• Eric:**   Estudo do problema, formulação inicial, estudo do Jupyter NB e do binder, formatação e escrita do relatório, codificação.<br>
**• Igor:**   Estudo do problema, formulação final, adoção de solução inicial, codificação, submissão do bot inicial para testes.<br>
**• Lucas:**  Estudo do problema, criação do repositório inicial, experienciar o game jogando e propor estratégias, codificação.
<br>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -<br>
**12/12/2018:** Definição dos membros da equipe e do problema a ser resolvido.<br>
**13/12/2018:** Primeiras impressões sobre o problema (estudo individual analítico sobre a mecânica e contexto histórico)<br>
**18/12/2018:** Primeiras impressões sobre o problema (estudo individual analítico sobre a mecânica e contexto histórico)<br>
**20/12/2018:** Hangout de debate sobre as implicações filosóficas e impacto científico-matemático do automato celular.<br>
**21/12/2018:** Recesso declarado oficial, todos sumiram.<br>
**07/01/2019:** Volta continua do recesso, os integrantes foram aparecendo aos poucos.<br>
**08/01/2019:** Encontro presencial. Brainstorm. Definição da abordagem e técnica de IA para resolver o problema.<br>
**09/01/2019:** Divisão do trabalho entre os membros da equipe.<br>
**10/01/2019:** Tentativa de formulação do problema usando a abordagem de busca local populacional, com algoritmos genéticos.<br>
**11/01/2019:** Tentativa de formulação do problema usando a abordagem de busca local populacional, com algoritmos genéticos.<br>
**14/01/2019:** Redefinição da abordagem para busca com adversários, através da técnica de Expectiminimax e tentativa de formulação.<br>
**14/01/2019:** Redefinição da abordagem, ainda em busca com adversários, porém através da técnica de Minimax e tentativa de formulação.<br>
**15/01/2019:** Formulação.<br>
**15/01/2019:** Estudo da plataforma Anaconda e Jupyter Notebook (tutoriais e minicursos virtuais).<br>
**15/01/2019:** Inicialização de documentação no Jupyter Notebook do relatório do projeto.<br>
**15/01/2019:** Formatação inicial do documento e descrição do problema de IA escolhido.<br>
**16/01/2019:** Melhor compreensão da API da plataforma riddles.io para o Game of Life and Death e download do starter bot disponibilizado pela mesma.<br>
**16/01/2019:** Teste de submissão do robô burro na plataforma riddles.io para competir com outros robôs.<br>
**16/01/2019:** Instalação do PyCharm IDE e criação do repositório inicial no GitHub para codificação da solução adotada.<br>
**16/01/2019:** Estudo da plataforma Binder para emular o servidor do Jupyter Notebook online (tutoriais).<br>
**16/01/2019:** Escrita do relatório.<br>
**17/01/2019:** Escrita do relatório.<br>
**17/01/2019:** Estudo dos códigos disponibilizados para a disciplina.<br>
**17/01/2019:** Implementação.<br>
**18/01/2019:** Estudo dos códigos disponibilizados para a disciplina.<br>
**18/01/2019:** Implementação.<br>
**19/01/2019:** Estudo dos códigos disponibilizados para a disciplina.<br>
**19/01/2019:** Implementação.<br>
**19/01/2019:** Teste de submissão da solução adotada na plataforma riddles.io para competir com outros robôs.<br>
**20/01/2019:** Estudo dos códigos disponibilizados para a disciplina.<br>
**20/01/2019:** Implementação.<br>
**20/01/2019:** Teste de submissão da solução adotada na plataforma riddles.io para competir com outros robôs.<br>
**21/01/2019:** Implementação, testes finais e entrada.<br>

### Avaliação final [RESTRITO AO PROFESSOR]

Critérios de avaliação do trabalho e pontuações:
1. (4pt) Adequabilidade da(s) técnica(s) ao problema (Justificativa);
2. (4pt) Implementação correta da(s) técnica(s) escolhida(s) (Corretude);
3. (2pt) Nível de similaridade com os códigos da disciplina (Reuso);
4. (10pt) Total = 1. + 2. + 3.
5. (+1pt) Extra: maior pontuação da turma (por jogo) (Competição)
6. Xpt = Pontos do integrante obtidos pela média da auto-avaliação da equipe (Participação)
7. Mpt = Média de auto-avaliação da equipe

**NOTA FINAL DO INTEGRANTE:** NF = Total + (Xpt - Mpt)